In [1]:
import pandas as pd
import sklearn as sk
import math 
from math import sqrt
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import time
from selenium import webdriver
from bs4 import BeautifulSoup
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Soumik\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Soumik\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [2]:
doc=[] #corpus
tokens_in_doc=[]


def __remove_punctuation(text):

        message = []
        for x in text:
            if x in string.punctuation:
                message.append(' ')
            else:
                message.append(x)
                
        message = ''.join(message)

        return message

def __remove_stopwords(text):

    words= []

    for x in text.split():

        x=x.lower()

        if x in stopwords.words('english'):
            pass
        
        elif x.isnumeric():
            pass

        else:
            # stemmer = nltk.stem.PorterStemmer()
            # x = stemmer.stem(x)
            lemma = nltk.wordnet.WordNetLemmatizer()   #lemmatization
            x=lemma.lemmatize(x)
            words.append(x)

    return words

def token_words(text):

    message = __remove_punctuation(text)
    words = __remove_stopwords(message)
    return words

In [3]:
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(ChromeDriverManager().install())

urls = []
titles = []


with open('lt_url.txt') as f:

    urls = f.readlines()

with open('lt_title.txt') as f:

    titles = f.readlines()



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [C:\Users\Soumik\.wdm\drivers\chromedriver\win32\101.0.4951.41\chromedriver.exe] found in cache
C:\Users\Soumik\AppData\Local\Temp\ipykernel_9700\3352578384.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [4]:
url_1=urls[0]

driver.get(url_1)

time.sleep(5)

html = driver.page_source

soup = BeautifulSoup(html, 'html.parser')

ques = soup.find("div", {"class": "content__u3I1 question-content__JfgR"})

problem_text_html = ques.findChild("div", recursive=False).encode('utf-8')

problem_text_html = problem_text_html.decode('ascii', 'ignore')

with open("problem_1.txt", "w+") as f:
    f.write(problem_text_html)

first_problem_text = ques.get_text()

first_problem_text = first_problem_text + " " + titles[0]

doc.append([first_problem_text])

first_problem_text = token_words(first_problem_text)

tokens_in_doc.append(first_problem_text)

total = set(first_problem_text)


In [5]:
cnt = 0

for url in urls:

    try:

        if(cnt==0):
            cnt+=1
            continue

        driver.get(url)

        time.sleep(5)


        html = driver.page_source

        soup = BeautifulSoup(html, 'html.parser')

        ques = soup.find("div", {"class": "content__u3I1 question-content__JfgR"})

        if(ques is None):
            print(cnt, url)
            titles.pop(cnt)
            continue

        problem_text_html = ques.findChild("div", recursive=False).encode('utf-8')

        problem_text_html = problem_text_html.decode('ascii', 'ignore')

        with open("problem_"+str(cnt+1)+".txt", "w+") as f:
            f.write(problem_text_html)

        problem_text = ques.get_text()

        problem_text = problem_text + " " + titles[cnt]
        
        doc.append([problem_text])

        problem_text = token_words(problem_text)

        tokens_in_doc.append(problem_text)

        total = total.union(set(problem_text))

        cnt += 1

    except:
        print(cnt)
        titles.pop(cnt)
        continue
    

652
727


In [6]:
wordDictA = dict.fromkeys(total, 0) 

for word in first_problem_text:
    wordDictA[word]+=1

df=pd.DataFrame([wordDictA])

df

,red,mean,priority,protected,mountain,ascii,orderly,shop,knight,deck,...,light,gnid,cnti,durationi,ghij,flood,principle,plank,false,attribute
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
cnt2=0

for each_doc in tokens_in_doc:
    
    if(cnt2==0):
        cnt2+=1
        continue

    wordDictB = dict.fromkeys(total, 0)
    for word in each_doc:
        wordDictB[word]+=1
    
    df = df.append([wordDictB], ignore_index=True)
    

In [8]:
def computeTF(wordDict, doc):
    tfDict = {}
    corpusCount = len(doc)
    for word, count in wordDict.items():
        tfDict[word] = count/float(corpusCount)
    return(tfDict)
#running our sentences through the tf function:

tf = pd.DataFrame([computeTF(df.iloc[0], tokens_in_doc[0])])

for i in range(1,cnt):
    tf_i = computeTF(df.iloc[i], tokens_in_doc[i])
    tf=tf.append([tf_i], ignore_index=True)    #Converting to dataframe for visualization

In [9]:
def computeIDF(data):

    idfDict = {}
    N = cnt
    
    idfDict = dict.fromkeys(data.keys(), 0)

    for key in data.keys():

        count= (df[key]>0).sum()

        if(count!=0):
            idfDict[key] = math.log10(N / (float(count)))   #if some  word occurs in all document, this will return 0, implying that it is least rare 
        else:
            idfDict[key] = 0
        
    return(idfDict)

idfs = computeIDF(df)

idf_df=pd.DataFrame([idfs])

idf_df

    

,red,mean,priority,protected,mountain,ascii,orderly,shop,knight,deck,...,light,gnid,cnti,durationi,ghij,flood,principle,plank,false,attribute
0,1.549718,1.367637,2.92993,2.92993,2.53199,2.189567,3.23096,2.53199,2.753838,2.92993,...,3.23096,3.23096,3.23096,3.23096,3.23096,2.92993,2.6289,3.23096,0.882655,2.753838


In [10]:
tf_idf = pd.DataFrame(tf.values*idf_df.values, columns=tf.columns, index=tf.index)

keywords = tf_idf.columns.values.tolist()

with open('keywords.txt', 'w', encoding="utf-8") as f:
    f.write(' '.join(map(str, keywords)))


In [11]:
magnitude=[]

def calculateMagnitude(v):
    magnitude = 0
    for j in range(0, len(tf_idf.columns)):
        magnitude += pow((v[j]), 2.0)
    return sqrt(magnitude * 1.0)

ls = tf_idf.values.tolist()

for i in range(0,cnt):
    magnitude.append(calculateMagnitude(ls[i]))

with open('magnitude.txt', 'w') as f:
    f.write(' '.join(map(str, magnitude)))

In [12]:
import numpy as np
arr = tf_idf.to_numpy()

for i in range(0,cnt):
    for j in range(0,len(tf_idf.columns)):
        if(arr[i][j]!=0):
            with open('tf_idf.txt', 'a') as f:
                f.write(str(i)+' '+str(j)+' '+str(arr[i][j])+'\n')


In [13]:
idf_df.to_csv(r'./idf.txt', header=None, index=None, sep=' ',mode='w')